In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [8]:
data = spark.read.csv('./data/SMSSpamCollection', inferSchema=True, sep='\t')

In [9]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [10]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [11]:
from pyspark.sql.functions import length 

In [12]:
data = data.withColumn('length', length(data['text']))

In [13]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [14]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [15]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [19]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [20]:
from pyspark.ml.feature import VectorAssembler

In [21]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'], outputCol='features')

In [22]:
from pyspark.ml.classification import NaiveBayes

In [23]:
nb = NaiveBayes()

In [24]:
from pyspark.ml import Pipeline

In [25]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])

In [26]:
cleaner = data_prep_pipe.fit(data)

In [27]:
clean_data = cleaner.transform(data)

In [28]:
clean_data = clean_data.select('label','features')

In [33]:
training,test = clean_data.randomSplit([0.7,0.3])

In [34]:
spam_detector = nb.fit(training)

In [35]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [37]:
test_results = spam_detector.transform(test)

In [38]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-812.24736163862...|[1.0,2.1714230380...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-823.01404567932...|[1.0,8.2367184739...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-535.28512395859...|[1.0,7.4356935111...|       0.0|
|  0.0|(13424,[0,1,14,79...|[-693.23676245223...|[1.0,4.8563911688...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-987.45384603376...|[0.99999999818543...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-750.24995736561...|[1.0,1.9044798995...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-1010.6222560654...|[1.0,1.2641875008...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1313.8649135391...|[1.0,3.7056848962...|       0.0|
|  0.0|(13424,[0,1,27,85...|[-1526.3697531037...|[1.97010943513813...|       1.0|
|  0.0|(13424,[0

In [39]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [40]:
acc_eval = MulticlassClassificationEvaluator()

In [41]:
acc = acc_eval.evaluate(test_results)

In [43]:
print('ACC of NB model')
print(acc)

ACC of NB model
0.9168810497165044
